In [1]:
import numpy as np 
import pandas as pd 

from sklearn.feature_extraction.text import CountVectorizer
from keras.layers import Dropout
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras.layers import Dense, Embedding, LSTM, concatenate, Input
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras import optimizers
import tensorflow as tf


import keras.backend as K

Using TensorFlow backend.


In [2]:
from utils import *
df = functions.parse_file(r"raw_data/EmoContext/train.txt", "EmoContext")
df.head(20)

,id,turn1,turn2,turn3,label
0,0,Don't worry I'm girl,hmm how do I know if you are,What's ur name?,others
1,1,When did I?,saw many times i think -_-,No. I never saw you,angry
2,2,By,by Google Chrome,Where you live,others
3,3,U r ridiculous,I might be ridiculous but I am telling the truth.,U little disgusting whore,angry
4,4,Just for time pass,wt do u do 4 a living then,Maybe,others
5,5,I'm a dog person,youre so rude,Whaaaat why,others
6,6,So whatsup,Nothing much. Sitting sipping and watching TV....,What are you watching on tv?,others
7,7,Ok,ok im back!!,"So, how are u",others
8,8,Really?,really really really really really,Y saying so many times...i can hear you,others
9,9,Bay,in the bay,😘 love you,others


In [3]:
df_test = functions.parse_file(r"testwithoutlabels.txt", "EmoContext")

In [4]:
text_data = []
for idx,row in df.iterrows():
    text_data.append(" {} fullstop {} fullstop {} fullstop".format(row['turn1'], row['turn2'], row['turn3']))

In [5]:
test_text_data = []
for idx,row in df_test.iterrows():
    test_text_data.append(" {} fullstop {} fullstop {} fullstop".format(row['turn1'], row['turn2'], row['turn3']))

In [6]:
emoticons_map = {
    '😘': ' emoticon',
    '😍': ' happyemoticon',
    '😁': ' happyemoticon',
    '😭': ' sademoticon',
    '😑': ' sademoticon',
    '😻': ' happyemoticon',
    '😂': ' happyemoticon',
    '👍': ' emoticon',
    '😀': ' happyemoticon',
    ':D': ' happyemoticon',
    '🙂':  ' happyemoticon',
    '<3': ' happyemoticon',
    '😓' : ' sademoticon',
    '😒' : ' angryemoticon',
    '😈' : ' emoticon',
    '👿' : ' angryemoticon',
    '🖑' : ' happyemoticon',
    '😾' : ' emoticon',
    '😠' : ' angryemoticon',
    '👻' : ' emoticon',
    ':(' : ' sademoticon',
    ':)' : ' happyemoticon',
    'xD' : ' happyemoticon',
    '💔' : ' sademoticon',
    '😥' : ' emoticon',
    '😞' : ' sademoticon',
    '😤' : ' angryemoticon',
    '😃' : ' happyemoticon',
    '😦' : ' sademoticon',
    ':3' : ' emoticon',
    '😼' : ' emoticon',
    '😏' : ' happyemoticon',
    '😱' : ' sademoticon',
    '😬' : ' sademoticon',
    '🙁' : ' sademoticon',
    '</3' : ' sademoticon',
    '😺' : ' happyemoticon',
    '😣' : ' angryemoticon',
    '😢' : ' sademoticon',
    '😆' : ' happyemoticon',
    '😄' : ' happyemoticon',
    '😅' : ' happyemoticon',
    ':-)' : ' happyemoticon',
    '😊' : ' happyemoticon',
    '😕' : ' sademoticon',
    '😽' : ' happyemoticon',
    '🙀' : ' angryemoticon',
    '🤣' : ' happyemoticon',
    '🤐' : ' emoticon',
    '😡' : ' sademoticon',
    '👌' : ' happyemoticon', 
    '😮' : ' emoticon',
    '❤️' : ' happyemoticon',
    '🙄' : ' happyemoticon',
    '😿' : ' sademoticon',
    '😉' : ' happyemoticon',
    '😋' : ' happyemoticon',
    '😐' : ' emoticon',
    '😹' : ' happyemoticon',
    '😴' : ' sademoticon',
    '💤' : ' emoticon',
    '😜' : ' happyemoticon',
    '😇' : ' happyemoticon',
    '😔' : ' sademoticon',
    '😩' : ' sademoticon',
    '❤' : ' happyemoticon',
    '😲' : ' emoticon',
    '😫' : ' sademoticon',
    '😳' : ' sademoticon',
    '😰' : ' sademoticon',
}

In [7]:
bad_words_df = pd.read_csv('bad-words.csv')
#bad_words_df['jigaboo']
bad_words = list(bad_words_df['jigaboo'])
bad_words.sort(key = lambda s: len(s))
bad_words = bad_words[::-1]

In [8]:
for i in range(0, len(text_data)):
    for k in emoticons_map.keys():
        if text_data[i].find(k) >= 0:
            text_data[i] = text_data[i].replace(k, emoticons_map[k])
    for bw in bad_words:
        bw = " {} ".format(bw)
        if text_data[i].find(bw) >= 0:
            text_data[i] = text_data[i].replace(bw, ' badword ')
            

In [9]:
for i in range(0, len(test_text_data)):
    for k in emoticons_map.keys():
        if test_text_data[i].find(k) >= 0:
            test_text_data[i] = test_text_data[i].replace(k, emoticons_map[k])
    for bw in bad_words:
        bw = " {} ".format(bw)
        if test_text_data[i].find(bw) >= 0:
            test_text_data[i] = test_text_data[i].replace(bw, ' badword ')
            

In [10]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from autocorrect import spell
#from spellchecker import SpellChecker
from nltk import pos_tag
from nltk.corpus import sentiwordnet as swn

porter = PorterStemmer()
lem = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
reserved_words = ['badword','fullstop','happyemoticon','sademoticon','angryemoticon']

#spell = SpellChecker()

def get_word_senti_score(word):
    good = list(swn.senti_synsets(word))
    posscore=0
    negscore=0
    for synst in good:

        posscore=posscore+synst.pos_score()
        negscore=negscore+synst.neg_score()

    posscore = posscore/len(good) if len(good)>0 else 0
    negscore = negscore/len(good) if len(good)>0 else 0
    
    return posscore, negscore


def filter_text(text):    
    text = text.lower()
    tokens = word_tokenize(text)
    words = [word for word in tokens if word.isalpha()]
    #words = [w for w in words if not w in stop_words]
    stemmed = [lem.lemmatize(word) if not word in reserved_words else word for word in tokens]
    corrected = [spell(word) if not word in reserved_words else word for word in stemmed ]
    corrected = [word.lower() for word in corrected]
    #corrected = [spell.correction(word) for word in stemmed]
    tagged = pos_tag(corrected)
    tagged = np.array(tagged)
    
    scores = []
    for word in corrected:
        posscore, negscore = get_word_senti_score(word)
        scores.append([posscore, negscore])
    
    return tagged, np.array(scores)

In [11]:
tokenized_train_data = []
tokenized_test_data = []
tagged_train_data = []
tagged_test_data = []
pos_scores_train_data = []
neg_scores_train_data = []
pos_scores_test_data = []
neg_scores_test_data = []



i=0
for sentence in text_data:
    taggs, scores = filter_text(sentence)
    tokenized_train_data.append(taggs[:,0])
    tagged_train_data.append(taggs[:,1])
    pos_scores_train_data.append(scores[:,0])
    neg_scores_train_data.append(scores[:,1])
    
    if i % 100 == 0:
        print(i)
    i+=1

i=0
for sentence in test_text_data:
    taggs, scores = filter_text(sentence)
    tokenized_test_data.append(taggs[:,0])
    tagged_test_data.append(taggs[:,1])
    pos_scores_test_data.append(scores[:,0])
    neg_scores_test_data.append(scores[:,1])
    if i % 100 == 0:
        print(i)
    i+=1

unique_tokenized_words = {}

for words in tokenized_train_data:
    for word in words:
        if word not in unique_tokenized_words:
            unique_tokenized_words[word] = 1
        else:
            unique_tokenized_words[word] += 1
            
for words in tokenized_test_data:
    for word in words:
        if word not in unique_tokenized_words:
            unique_tokenized_words[word] = 1
        else:
            unique_tokenized_words[word] += 1

sorted_unique_tokenized_words = sorted(unique_tokenized_words.items(), key=lambda kv: kv[1], reverse=True)
token_values = {}
i = 1
for word in sorted_unique_tokenized_words:
    token_values[word[0]] = i
    i += 1

final_tokenized_train_data = []
for i in range(0,len(tokenized_train_data)):
    sentence = []
    for word in tokenized_train_data[i]:
        sentence.append(token_values[word])
    final_tokenized_train_data.append(sentence)

final_tokenized_test_data = []
for i in range(0,len(tokenized_test_data)):
    sentence = []
    for word in tokenized_test_data[i]:
        sentence.append(token_values[word])
    final_tokenized_test_data.append(sentence)


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [12]:

unique_tagged_words = {}

for words in tagged_train_data:
    for word in words:
        if word not in unique_tagged_words:
            unique_tagged_words[word] = 1
        else:
            unique_tagged_words[word] += 1
            
for words in tagged_test_data:
    for word in words:
        if word not in unique_tagged_words:
            unique_tagged_words[word] = 1
        else:
            unique_tagged_words[word] += 1

sorted_unique_tagged_words = sorted(unique_tagged_words.items(), key=lambda kv: kv[1], reverse=True)
tagged_values = {}
i = 1
for word in sorted_unique_tagged_words:
    tagged_values[word[0]] = i
    i += 1

final_tagged_train_data = []
for i in range(0,len(tagged_train_data)):
    sentence = []
    for word in tagged_train_data[i]:
        sentence.append(tagged_values[word])
    final_tagged_train_data.append(sentence)

final_tagged_test_data = []
for i in range(0,len(tagged_test_data)):
    sentence = []
    for word in tagged_test_data[i]:
        sentence.append(tagged_values[word])
    final_tagged_test_data.append(sentence)

In [13]:
len(unique_tokenized_words)

13189

In [14]:
def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

In [15]:
NR_WORDS1 = len(unique_tokenized_words)+1
NR_WORDS2 = len(unique_tagged_words)+1

In [16]:
np.save("final_tokenized_train_data.npy", final_tokenized_train_data)
np.save("final_tokenized_test_data.npy", final_tokenized_test_data)
np.save("final_tagged_train_data.npy", final_tagged_train_data)
np.save("final_tagged_test_data.npy", final_tagged_test_data)
np.save("pos_scores_train_data.npy", pos_scores_train_data)
np.save("pos_scores_test_data.npy", pos_scores_test_data)
np.save("neg_scores_train_data.npy", neg_scores_train_data)
np.save("neg_scores_test_data.npy", neg_scores_test_data)

In [17]:
final_tokenized_train_data = np.load("final_tokenized_train_data.npy")
final_tokenized_test_data = np.load("final_tokenized_test_data.npy")
final_tagged_train_data = np.load("final_tagged_train_data.npy")
final_tagged_test_data = np.load("final_tagged_test_data.npy")
pos_scores_train_data = np.load("pos_scores_train_data.npy")
pos_scores_test_data = np.load("pos_scores_test_data.npy")
neg_scores_train_data = np.load("neg_scores_train_data.npy")
neg_scores_test_data = np.load("neg_scores_test_data.npy")

In [18]:
X1 = pad_sequences(final_tokenized_train_data)
X2 = pad_sequences(final_tagged_train_data)
X3 = pad_sequences(pos_scores_train_data)
X4 = pad_sequences(neg_scores_train_data)

In [19]:
words = {"others": 0, "angry": 1, "sad":2, "happy": 3}

def one_hot_vector(word):
    y = [0,0,0,0]
    y[words[word]] = 1
    return y

Y = []

for idx,row in df.iterrows():
    Y.append(one_hot_vector(row['label']))

Y = np.array(Y)

In [20]:
NR_WORDS1

13190

In [21]:
NR_WORDS2

39

#### model

In [70]:
embed_dim = 128
lstm_out = 64
batch_size = 128

model1_input = Input(shape=(X1.shape[1],))
model1 = Embedding(NR_WORDS1, 256, input_length = X1.shape[1])(model1_input)
model1 = LSTM(lstm_out)(model1)
model2_input = Input(shape=(X2.shape[1],))
model2 = Embedding(NR_WORDS2, 64, input_length = X2.shape[1])(model2_input)
model2 = LSTM(lstm_out)(model2)

model3_input = Input(shape=(X3.shape[1],))
model3 = Dense(64, activation='sigmoid')(model3_input)
model4_input = Input(shape=(X4.shape[1],))
model4 = Dense(64, activation='sigmoid')(model4_input)

merged = concatenate([model1, model2])
merged = Dense(64,activation='relu')(merged)
merged = Dropout(0.3)(merged)
merged = Dense(4,activation='softmax')(merged)

model = Model(inputs=[model1_input, model2_input], outputs=merged)

#model = LSTM(lstm_out)(model)
#model.add(Dropout(0.3))
#model.add(Dense(4,activation='softmax'))
adam = optimizers.Adam(lr=0.01)
rmsprop = optimizers.RMSprop(lr=0.005, rho=0.9, epsilon=None, decay=0.0)
model.compile(loss = 'binary_crossentropy', optimizer=rmsprop, metrics = ['accuracy', f1])

In [71]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_17 (InputLayer)           (None, 164)          0                                            
__________________________________________________________________________________________________
input_18 (InputLayer)           (None, 164)          0                                            
__________________________________________________________________________________________________
embedding_9 (Embedding)         (None, 164, 256)     3376640     input_17[0][0]                   
__________________________________________________________________________________________________
embedding_10 (Embedding)        (None, 164, 64)      2496        input_18[0][0]                   
__________________________________________________________________________________________________
lstm_9 (LS

In [72]:
#X_train_1, X_test_1, Y_train, Y_test = train_test_split(X1,Y, test_size = 0, random_state = 42)
#X_train_2, X_test_2, Y_train_2, Y_test_2 = train_test_split(X2,Y, test_size = 0, random_state = 42)

In [73]:
#model.fit([X1, X2, X3, X4], Y, batch_size = 128, nb_epoch = 3, verbose = 1)

In [74]:
#K.set_value(model.optimizer.lr, 0.0001)

In [75]:
from sklearn.model_selection import KFold
# prepare cross validation
kfold = KFold(n_splits=4)
Y = np.array(Y)
# enumerate splits
for train, validation in kfold.split(X1):
    history = model.fit([X1[train], X2[train]], Y[train],
                    validation_data=([X1[validation], X2[validation]], Y[validation]),
                    epochs=1, verbose=1, batch_size=128)

Train on 22620 samples, validate on 7540 samples
Epoch 1/1
22620/22620 [==============================] - 161s 7ms/step - loss: 0.2771 - acc: 0.8916 - f1: 0.7221 - val_loss: 0.2189 - val_acc: 0.9089 - val_f1: 0.7979
Train on 22620 samples, validate on 7540 samples
Epoch 1/1
22620/22620 [==============================] - 161s 7ms/step - loss: 0.1812 - acc: 0.9304 - f1: 0.8466 - val_loss: 0.1639 - val_acc: 0.9360 - val_f1: 0.8573
Train on 22620 samples, validate on 7540 samples
Epoch 1/1
22620/22620 [==============================] - 161s 7ms/step - loss: 0.1484 - acc: 0.9446 - f1: 0.8776 - val_loss: 0.1450 - val_acc: 0.9456 - val_f1: 0.8806
Train on 22620 samples, validate on 7540 samples
Epoch 1/1
22620/22620 [==============================] - 163s 7ms/step - loss: 0.1250 - acc: 0.9538 - f1: 0.8966 - val_loss: 0.1174 - val_acc: 0.9560 - val_f1: 0.9024


#### Evaluation

In [27]:
ev_result = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("Score: %.3f" % (ev_result[0]))
print("Validation Accuracy: %.3f" % (ev_result[1]))
print("F1 score: %.3f" % (ev_result[2]))

NameError: name 'X_test' is not defined

In [76]:
model_json = model.to_json()
with open("multiple_features.json", "w") as outfile:
    outfile.write(model_json)
model.save_weights("multiple_features.h5")

#### For submission

In [77]:
X1_test = pad_sequences(final_tokenized_test_data, maxlen=X1.shape[1])
X2_test = pad_sequences(final_tagged_test_data, maxlen=X2.shape[1])
X3_test = pad_sequences(pos_scores_test_data, maxlen=X3.shape[1])
X4_test = pad_sequences(neg_scores_test_data, maxlen=X4.shape[1])

In [78]:
res = model.predict([X1_test, X2_test], batch_size=128, verbose=1)

5509/5509 [==============================] - 10s 2ms/step


In [79]:
revers_words = {0:"others", 1:"angry", 2:"sad", 3:"happy"}

def softmax_convert(res):
    max_i = 0
    max_v = 0
    for i in range(0,4):
        if res[i] > max_v:
            max_v = res[i]
            max_i = i
    return revers_words[max_i]

In [80]:
results = []
for r in res:
    results.append(softmax_convert(r))

In [81]:
df_test['label'] = results

In [82]:
df_test.head(30)

,id,turn1,turn2,turn3,label
0,0,Hmm,What does your bio mean?,I don’t have any bio,others
1,1,What you like,very little things,Ok,others
2,2,Yes,How so?,I want to fuck babu,angry
3,3,what did you guess,what what,fuck,others
4,4,We ?,of course we will!,What gender movies you like??,others
5,5,Where are you now?,At home just about to have breakfast...,what are you eating?,others
6,6,That was a joke btw...,it was,Yes 😁,happy
7,7,Who d hell s he,johnny depp...duh?,Who she,angry
8,8,"yes, good advice",best advice ...,i great thx,others
9,9,Nice to meet u,"Hi, nice to meet you too! 😸😂",😁,happy


In [83]:
df_test.to_csv("test.txt",index=False , sep="\t")

In [84]:
res_bidirectional = np.load("res_bidirectional.npy")

